# Week 4: Retrieval-Augmented Generation (RAG) with arXiv Papers
This week marks a major shift in your AI agent's capabilities: you’ll build the foundation for a Retrieval-Augmented Generation (RAG) system tailored to scientific research. Rather than relying on an LLM’s memory alone, RAG architectures allow your agent to search a structured knowledge base and generate grounded, document-aware answers.

Your task is to create a RAG pipeline using recent arXiv cs.CL papers, converting them into searchable chunks, embedding them, and indexing them with FAISS. You’ll then implement a simple query interface that takes a user question, retrieves the top relevant chunks, and displays them for further processing.

This week marks the beginning of building your agent’s private research knowledge base—a semantic index that you’ll evolve into a full-featured hybrid database in Week 5.


## 📚 Learning Objectives

* Understand the components of a Retriever-Reader QA pipeline.
* Explore document chunking strategies (e.g., sections vs. sliding windows) and their impact on retrieval performance.
* Index scientific text using vector embeddings and FAISS.
* Build and query a semantic index via a FastAPI endpoint that returns relevant passages.


## Project Design

The project will guide you through building a RAG pipeline on arXiv cs.CL papers:

1. **Data Collection:** Obtain 50 arXiv cs.CL PDFs (you can scrape via the arXiv API or use a provided sample set).
2. **Text Extraction:** Extract raw text from each PDF (for example, using PyMuPDF's `get_text()` on each page). Clean and concatenate the page text into full-document strings.
3. **Text Chunking:** Split each paper into chunks (≤ 512 tokens each). You might split at section boundaries or use a sliding-window approach (e.g., 500-token windows with overlap). Chunking into smaller, meaningful segments (around 250–512 tokens) often yields better retrieval precision.
4. **Embedding Generation:** Compute dense vector embeddings for each chunk. For instance, using the `sentence-transformers` library:

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(list_of_chunks)  # embeds each text chunk into a 384-d vecto

   (Alternatively, you can use a Hugging Face Transformer model and apply pooling manually to get chunk embeddings.)
5. **Indexing with FAISS:** Build a FAISS index of the chunk embeddings. For example, use a simple index like `IndexFlatL2` with the same dimensionality as your embeddings. Add all chunk vectors to the index (e.g., `index.add(np.array(embeddings))`).
6. **Notebook Demo:** Create a notebook where a user query is embedded and passed to the index (`index.search(query_embedding, k)`) to retrieve the top-3 matching chunks. Display the original chunk text for these results.
7. **FastAPI Service:** Build a simple FastAPI app. Define an endpoint (e.g. `@app.get("/search")`) that accepts a query parameter `q`. In the handler, embed `q`, perform the FAISS search, and return the top passages as JSON. (For example, a FastAPI endpoint can accept a question and return relevant documents.)



## Starter Code Snippets

Below are skeleton code templates. Fill in the details (indicated by comments or ellipses).

**Data Extraction (PDF → Text):**

In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Open a PDF and extract all text as a single string.
    """
    doc = fitz.open(pdf_path)
    pages = []
    for page in doc:
        page_text = page.get_text()  # get raw text from page
        # (Optional) clean page_text here (remove headers/footers)
        pages.append(page_text)
    full_text = "\n".join(pages)
    return full_text


In [4]:
text = extract_text_from_pdf('python-developer-resume-example.pdf')

**Chunking Logic (Sliding Window):**

In [15]:
from typing import Iterable, List, Tuple, Callable
def chunk_text(text: str, max_tokens: int = 100, overlap: int = 10) -> List[str]:
    tokens = text.split()
    chunks = []
    step = max_tokens - overlap
    for i in range(0, len(tokens), step):
        chunk = tokens[i:i + max_tokens]
        chunks.append(" ".join(chunk))
    return chunks


In [16]:
chunks = chunk_text(text)


**Embedding Generation (Sentence-Transformers):**

In [18]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks) 


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



**FAISS Indexing and Search:**


In [34]:

import faiss
import numpy as np

# Assume embeddings is a 2D numpy array of shape (num_chunks, dim)
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)  # using a simple L2 index
index.add(np.array(embeddings))  # add all chunk vectors

# Example: search for a query embedding
query_embedding = model.encode(["What are the key skills of the candidate?"])[0]  # get embedding for the query (shape: [1, dim])
k = 3
distances, indices = index.search(np.array([query_embedding]), k)
# indices[0] holds the top-k chunk indices




**FastAPI Route Skeleton:**


In [ ]:

from fastapi import FastAPI
import numpy as np
import os

app = FastAPI()

@app.get("/search")
async def search(q: str):
    """
    Receive a query 'q', embed it, retrieve top-3 passages, and return them.
    """
    # TODO: Embed the query 'q' using your embedding model
    query_vector = model.encode([q])[0]   # e.g., model.encode([q])[0]
    # Perform FAISS search
    k = 3
    distances, indices = index.search(np.array([query_vector]), k)
    # Retrieve the corresponding chunks (assuming 'chunks' list and 'indices' shape [1, k])
    results = []
    for idx in indices[0]:
        results.append(chunks[idx])
    return {"query": q, "results": results}

if __name__ == "__main__":
    import uvicorn
    
    host = os.getenv("HOST", "0.0.0.0")
    port = int(os.getenv("PORT", 8000))
    debug = os.getenv("DEBUG", "True").lower() == "true"
    
    uvicorn.run(
        "main:app",
        host=host,
        port=port,
        reload=debug,
        log_level="info"
    )


INFO:     Will watch for changes in these directories: ['/Users/yunfeibai/Desktop/GenAI MLE Course/MLE_in_GenAI-Course/class4']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [36875] using WatchFiles
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR:    Error loading ASGI app. Could not import module "main".
INFO:     Stopping reloader process [36875]



## Deliverables

* **Code Notebook / Script:** Complete code for the RAG pipeline (PDF extraction, chunking, embedding, indexing, retrieval).
* **Data & Index:** The FAISS index file and the set of 50 processed paper chunks (e.g., as JSON or pickled objects).
* **Retrieval Report:** A brief report showing at least 5 example queries and the top-3 retrieved passages for each, to demonstrate system performance.
* **FastAPI Service:** The FastAPI app code (e.g. `main.py`) and instructions on how to run it. The `/search` endpoint should be demonstrable (e.g. returning top-3 passages in JSON for sample queries).

## Student Exploration Tips

* Experiment with different chunk sizes and overlaps. Smaller chunks (∼250 tokens) often give more precise retrieval, while larger chunks include more context.
* Try different embedding models (e.g. using `'all-mpnet-base-v2'` or `'paraphrase-MiniLM-L6-v2'`) to see how retrieval results change.
* Implement a simple reranking step: for example, after retrieving candidates with FAISS, re-score them with a cross-encoder model for finer ranking.
* Use metadata: consider filtering or weighting chunks by paper metadata (e.g. year, authors, keywords) to improve relevance if needed.


# Below Codes for generate deliverables files

In [ ]:
# Codes for generate deliverables files
import fitz  # PyMuPDF
import requests
import faiss
import numpy as np
import requests
import xml.etree.ElementTree as ET
from sentence_transformers import SentenceTransformer
from typing import List
import uvicorn
from fastapi import FastAPI
import numpy as np
import os

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Open a PDF and extract all text as a single string.
    """
    if pdf_path.startswith("http://") or pdf_path.startswith("https://"):
        response = requests.get(pdf_path)
        doc = fitz.open(stream=response.content, filetype="pdf")
    else:
        doc = fitz.open(pdf_path)
    pages = []
    for page in doc:
        page_text = page.get_text()  # get raw text from page
        # (Optional) clean page_text here (remove headers/footers)
        pages.append(page_text)
    full_text = "\n".join(pages)
    return full_text

def chunk_text(text: str, max_tokens: int = 500, overlap: int = 50) -> List[str]:
    tokens = text.split()
    chunks = []
    step = max_tokens - overlap
    for i in range(0, len(tokens), step):
        chunk = tokens[i:i + max_tokens]
        chunks.append(" ".join(chunk))
    return chunks

# load pdfs and extract text
def query_arxiv(category, max_results=50):  # Limited to 50 for size; change to 200 if needed
    url = f"http://export.arxiv.org/api/query?search_query=cat:{category}&sortBy=submittedDate&sortOrder=descending&max_results={max_results}"
    response = requests.get(url)
    root = ET.fromstring(response.content)
    entries = []
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        pdf_url = entry.find('{http://www.w3.org/2005/Atom}id').text.replace('abs', 'pdf') + '.pdf'
        paper = {
            'pdf_url': pdf_url,
            'title': entry.find('{http://www.w3.org/2005/Atom}title').text.strip(),
        }
        entries.append(paper)
    return entries

# Embed chunks using SentenceTransformer
def embed_chunks(chunks: List[str], model: SentenceTransformer) -> List[List[float]]:
     
     embeddings = model.encode(chunks)
     return embeddings

# Assume embeddings is a 2D numpy array of shape (num_chunks, dim)
def create_faiss_index(embeddings: List[float]) -> faiss.Index: 
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)  # using a simple L2 index
    index.add(np.array(embeddings))  # add all chunk vectors
    return index

# Initialize model, chunks, and index at module level
# process multiple arxiv papers
arxiv_papers = query_arxiv('cs.AI', max_results=10)  # e.g., most 10 papers from AI research
texts = []
for paper in arxiv_papers:
    text = extract_text_from_pdf(paper['pdf_url'])
    texts.append(text)
full_text = "\n".join(texts)

# setting up embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# chunking and embedding
chunks = chunk_text(full_text)
embeddings = embed_chunks(chunks, model)
index = create_faiss_index(np.array(embeddings))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Deliverables 2: Data and Index

# Save FAISS index (binary format)
faiss.write_index(index, "index and chunks examples/faiss_index.bin")
# Load FAISS index
# index = faiss.read_index("faiss_index.bin")

# Save chunks as JSONL with 50 examples
import json
chunks_count = len(chunks)
if chunks_count < 50:
    raise ValueError(f"Not enough chunks to save 50 examples, only found {chunks_count} chunks.")
else:
    with open("index and chunks examples/chunks_50_examples.json", "w") as f:
        for i in range(50):
            chunk = {"chunk_index": i, "chunk_text": chunks[i]}
            json.dump(chunk, f)
            f.write("\n")

In [18]:
# Deliverables 3: Retrieval report
"""
Receive a query 'q', embed it, retrieve top-3 passages, and return them.
"""
querys =    ["What are the latest advancements in AI research?",
            "Summarize the key techniques used in machine learning.",
            "Explain the concept of reinforcement learning.",
            "What are the challenges in natural language processing?",
            "Describe the applications of computer vision."]
results = []
for query in querys:
    query_vector = model.encode([query])[0]   # e.g., model.encode([query])[0]
    # Perform FAISS search
    k = 3  # top-k results
    distances, indices = index.search(np.array([query_vector]), k)
    # Retrieve the corresponding chunks (assuming 'chunks' list and 'indices' shape [1, k])
    results.append(f"Query: {query}\n")
    for i,idx in enumerate(indices[0]):
        results.append(f"top-{i+1} chunk: {chunks[idx]}\n\n")
with open("retrieval_report.json", "w") as f:
    json.dump({"retrieval_report": results}, f, indent=4)